In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from math import sqrt
import matplotlib.pyplot as plt
import seaborn as sns

####################################
# 커스텀 Transformer Encoder Layer (어텐션 가중치 반환)
####################################
class CustomTransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1, activation="relu"):
        super(CustomTransformerEncoderLayer, self).__init__()
        self.self_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout)
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, d_model)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.activation = nn.ReLU() if activation=="relu" else nn.GELU()
        
    def forward(self, src, src_mask=None, src_key_padding_mask=None):
        # need_weights=True로 어텐션 가중치를 반환합니다.
        attn_output, attn_weights = self.self_attn(
            src, src, src, attn_mask=src_mask, key_padding_mask=src_key_padding_mask, need_weights=True)
        src2 = self.dropout1(attn_output)
        src = self.norm1(src + src2)
        src2 = self.linear2(self.dropout(self.activation(self.linear1(src))))
        src = self.norm2(src + src2)
        return src, attn_weights

####################################
# 커스텀 Transformer Encoder (여러 레이어)
####################################
class CustomTransformerEncoder(nn.Module):
    def __init__(self, encoder_layer, num_layers):
        super(CustomTransformerEncoder, self).__init__()
        self.layers = nn.ModuleList([encoder_layer for _ in range(num_layers)])
    
    def forward(self, src, mask=None, src_key_padding_mask=None):
        attn_weights_all = []
        output = src
        for layer in self.layers:
            output, attn_weights = layer(output, src_mask=mask, src_key_padding_mask=src_key_padding_mask)
            attn_weights_all.append(attn_weights)  # 각 레이어의 어텐션 가중치 저장
        return output, attn_weights_all

####################################
# 기존 전처리, Dataset 등은 그대로 사용 (생략)
####################################
# (여기서는 기존 코드의 rolling_minmax_scale, bin_and_encode, TimeSeriesDataset 등 그대로 둡니다.)

####################################
# 모델 정의 (커스텀 Encoder 사용)
####################################
class EncoderOnlyTransformerCustom(nn.Module):
    def __init__(self, input_dim, embedding_dim=512, num_layers=6, nhead=8,
                 ffn_dim=2048, num_classes=2, max_seq_len=24):
        super(EncoderOnlyTransformerCustom, self).__init__()
        self.token_embedding = nn.Linear(input_dim, embedding_dim)
        self.position_embedding = nn.Embedding(max_seq_len, embedding_dim)
        # 커스텀 Transformer Encoder 사용
        encoder_layer = CustomTransformerEncoderLayer(d_model=embedding_dim, nhead=nhead, dim_feedforward=ffn_dim)
        self.transformer_encoder = CustomTransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(embedding_dim, num_classes)
        # forward 실행 후 어텐션 가중치를 저장할 변수
        self.last_attn_weights = None

    def forward(self, x):
        batch_size, seq_len, _ = x.shape
        x = self.token_embedding(x)
        positions = torch.arange(seq_len, device=x.device).unsqueeze(0).expand(batch_size, seq_len)
        pos_emb = self.position_embedding(positions)
        x = x + pos_emb
        x = x.transpose(0, 1)  # [seq_len, batch, embedding_dim]
        output, attn_weights_all = self.transformer_encoder(x)
        self.last_attn_weights = attn_weights_all  # 전체 레이어의 어텐션 가중치 저장
        return self.fc(output[-1, :, :])  # 마지막 타임스탭의 출력으로 분류 예측

####################################
# 어텐션 맵 시각화 함수
####################################
def plot_attention_map(attn_map, head_idx=0):
    plt.figure(figsize=(8, 6))
    sns.heatmap(attn_map[0, head_idx], cmap="viridis", annot=False)
    plt.xlabel("Key")
    plt.ylabel("Query")
    plt.title(f"Attention Map (Head {head_idx})")
    plt.show()

####################################
# 예시: 모델 실행 후 어텐션 가중치 확인하기
####################################
# 예를 들어, 백테스팅용 DataLoader에서 한 배치를 가져와 모델에 넣고 어텐션 가중치를 출력해봅니다.
# (아래는 단독 실행 예시이며, 실제 코드에 맞게 DataLoader나 입력 데이터를 구성해야 합니다.)

# 가상의 입력 (batch_size=2, seq_len=24, input_dim=400) 생성
dummy_input = torch.randn(2, 24, 400)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dummy_input = dummy_input.to(device)

# 모델 생성 (입력 차원 400, 분류 2개)
model = EncoderOnlyTransformerCustom(input_dim=400).to(device)
model.eval()

# forward 실행
with torch.no_grad():
    outputs = model(dummy_input)
    # 이제 model.last_attn_weights는 리스트로 각 레이어의 어텐션 가중치를 포함합니다.
    # 예: 첫 번째 레이어의 어텐션 가중치 확인 (shape: [batch, num_heads, seq_len, seq_len])
    first_layer_attn = model.last_attn_weights[0]
    print("첫 번째 레이어 어텐션 가중치 shape:", first_layer_attn.shape)
    # 시각화 (예: 첫 번째 헤드)
    plot_attention_map(first_layer_attn, head_idx=0)


ModuleNotFoundError: No module named 'matplotlib'